<a href="https://colab.research.google.com/github/springboardmentor0327/Text_Summarization_Infosys_Internship_Oct2024/blob/Sameer/LLM_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [2]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain langchain-google-genai langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 314.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

def load_llm(model="gemini-1.5-pro"):

  if model == "gemini-1.5-pro":
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2)
    return llm
  elif model == "gemini-1.5-flash":
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2)
    return llm
  else:
    raise ValueError("Invalid model name")

In [7]:
from langchain_core.prompts import ChatPromptTemplate

def get_prompt_template():
    # Define prompt with a strict word range instruction
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Write a concise summary of the following in {min_word_count} to {max_word_count} words. Stop once you reach the word limit:\n\n",
            ),
            ("human", "{context}")
        ]
    )
    return prompt

def llm_model(text, max_word_count=50):
    model="gemini-1.5-pro"
    # Set default for min_word_count if not provided
    # if min_word_count is None:
    min_word_count = max_word_count - 5

    # Load LLM
    llm = load_llm(model)

    # Get Prompt Template
    prompt = get_prompt_template()

    # Instantiate chain
    chain = prompt | llm

    # Invoke chain with specified range
    result = chain.invoke({
        "context": text,
        "min_word_count": min_word_count,
        "max_word_count": max_word_count
    })

    # Process output to enforce word limit
    output = result.content
    words = output.split()

    # If output exceeds max_word_count, truncate to max_word_count
    if len(words) > max_word_count:
        output = ' '.join(words[:max_word_count]) + "."

    # Return truncated output if necessary
    return output,(f"\nWord Count (Summary): {len(output.split())}")


In [8]:
text = '''
Text summarization involves creating a summary of a source text using natural language processing.
This is useful for condensing long-form text, audio, or video into a shorter, more digestible form that still conveys the main points.
Examples include news articles, scientific papers, podcasts, speeches, lectures, and meeting recordings.

There are two main types of summarization:

Extractive summarization: This type identifies and extracts key phrases or sentences (i.e., excerpts) from the source text and combines them into a summary.
It leaves the original text unchanged and only selects the important parts.

Abstractive summarization: This type involves understanding the main ideas in the source text and creating a new summary that expresses those ideas in a fresh and condensed way (i.e., paraphrasing).
 It's more complex because it requires a deeper understanding of the source text and the ability to convey the same information in fewer words.
'''

# Specifcy the text and number of words
summary = summarize_text(text, max_word_count = 50, model = "gemini-1.5-flash")


print(f"Original Text: {text}")
print(f"\n Text Length: {len(text)}")
print("="*100)
print(f"\nSummary: {summary}")
print(f"\nSummary Length: {len(summary)}")

Original Text: 
Text summarization involves creating a summary of a source text using natural language processing.
This is useful for condensing long-form text, audio, or video into a shorter, more digestible form that still conveys the main points.
Examples include news articles, scientific papers, podcasts, speeches, lectures, and meeting recordings.

There are two main types of summarization:

Extractive summarization: This type identifies and extracts key phrases or sentences (i.e., excerpts) from the source text and combines them into a summary.
It leaves the original text unchanged and only selects the important parts.

Abstractive summarization: This type involves understanding the main ideas in the source text and creating a new summary that expresses those ideas in a fresh and condensed way (i.e., paraphrasing).
 It's more complex because it requires a deeper understanding of the source text and the ability to convey the same information in fewer words.


 Text Length: 962

Su